In [1]:
import re  # 正規表現ライブラリをインポート

def validate_and_correct_tnm_output(tnm_stage):
    """
    TNM分類の形式を検証し、不正な形式の場合は修正する。
    """
    # 正規表現でTNM分類をチェック
    tnm_pattern = (
        r"T(?:0|is|1mi|1[abc]?|2[ab]?|3|4) "  # Tの分類
        r"N(?:0|1|2|3) "                     # Nの分類
        r"M(?:0|1[abc]?)"                    # Mの分類
    )
    if re.fullmatch(tnm_pattern, tnm_stage):
        # 正しい形式の場合、そのまま返す
        return tnm_stage
    else:
        # 正規表現パターンに一致する部分を検索
        match = re.search(tnm_pattern, tnm_stage)
        if match:
            return match.group()  # 一致する部分文字列を返す
        else:
            return None  # 一致する部分がない場合はNoneを返す

tmp = 'output: T1c N0 M0'
print(validate_and_correct_tnm_output(tmp))

T1c N0 M0


In [15]:
import pandas as pd

train_label_path = '../radnlp_2024_train_val_20240731/ja/main_task/train/label.csv'
df_train_label = pd.read_csv(train_label_path)

few_shot_examples = []

def example(row):
    ex_dict = {}
    study_id = row["id"]
    t = row["t"]
    n = row["n"]
    m = row["m"]
    txt_path = f'../radnlp_2024_train_val_20240731/ja/main_task/train/{study_id}.txt'
    with open(txt_path, 'r', encoding='utf-8') as file:
        content = file.read()
    ex_dict["文章"] = content
    ex_dict["出力"] = f"{t} {n} {m}"
    few_shot_examples.append(ex_dict)

df_train_label.apply(example, axis=1)

#display(df_train_label.head())
few_shot_examples[:2]

[{'文章': '左上葉全体が無気肺になっています。\n左上葉気管支は閉塞して造影  CT  で増強効果の乏しい 74mm  の腫瘤があります。\n肺癌と考えます。\n左肺門、同側縦隔リンパ節腫大しリンパ節転移と考えます。\n気管右側にもリンパ節腫大があり、こちらもリンパ節転移を疑います。\n左下葉気管支も腫瘍により浸潤あり、狭窄しています。\n胸水貯留は認めません。\n撮影範囲の腹部臓器に粗大な異常を認めません。',
  '出力': 'T4 N3 M0'},
 {'文章': '右下葉 S8 に⻑径 30mm の腫瘤性病変があります。辺縁には spicula を伴っています。肺癌の可能性がありますので、呼吸器科的に精査ください。\n左下葉には炎症瘢痕様の索状影を認めます。\n右肺門リンパ節腫大が疑われます。胸水認めません。',
  '出力': 'T1c N0 M0'}]

In [20]:
%%time

import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import pandas as pd
from dotenv import load_dotenv
from langchain.callbacks import tracing_enabled

import os
import pandas as pd
from langchain import PromptTemplate, FewShotPromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

from langchain_anthropic import ChatAnthropic

from langsmith import Client, traceable

import datetime
import pytz
now = str(datetime.datetime.now(pytz.timezone('Asia/Tokyo')))[:-16]; now = now.replace(" ", "_").replace("-", "_").replace(":", "_")

load_dotenv()
ANTHROPIC_API_KEY  = os.getenv('ANTHROPIC_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

# OpenAIのAPIキーを環境変数から取得
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = f"{now}_radnlp2024"
model_names = ['o1-preview-2024-09-12']  # ['gpt-4o-2024-05-13','gpt-4o-mini-2024-07-18','o1-preview-2024-09-12']
#model_names = ["claude-3-sonnet-20240229"]

# プロンプトテンプレートの読み込み
with open('../tnm_prompt.txt', 'r', encoding='utf-8') as file:
    tnm_prompt_text = file.read()

# FewShotPromptTemplateを作成
examples = [{"content": ex["文章"], "tnm": ex["出力"]} for ex in few_shot_examples]

example_template = """
文章: {content}
出力: {tnm}
"""

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=PromptTemplate(
        input_variables=["content", "tnm"],
        template=example_template
    ),
    prefix=(
        "あなたは優秀な医師です。以下の文章に基づき肺癌に関して常に正しい判断ができます。"
        "進行度分類は以下のTNM第８版に準拠しています。何も言わずに以下を覚え、与えられた文章からよく考えてTNM分類を選んでください。\n\n"
        f"{tnm_prompt_text}"
    ),
    suffix=(
        "以下の文章を読んで、TNM分類を正確に選択し、必ず以下の形式で出力してください：\n"
        "T<number>[optional_letter] N<number>[optional_letter] M<number>[optional_letter]\n\n"
        "{content}\n\n"
        "出力："
    ),
    input_variables=["content"]
)
# モデルごとに処理
for model_name in model_names:
    print(f"モデル {model_name} を使用してTNM分類を予測中...")
    output_csv = f'../fewshot_submission_{model_name}.csv'

    if "claude" in model_name:
        llm = ChatAnthropic(
            model=model_name,
            temperature=0,
        )
    # LLMの設定（ChatGPTを使用）
    elif '4o' in model_name:
        llm = ChatOpenAI(temperature=0.7,model_name=model_name)
    else:
        llm = ChatOpenAI(temperature=1,model_name=model_name)

    # チェーンの作成
    chain = LLMChain(llm=llm, prompt=few_shot_prompt)

    # テスト用の入力
    txt_folder = '../radnlp_2024_train_val_20240731/ja/main_task/val/'  # ここをTXTファイルが保存されているフォルダに変更
    txt_files = [f for f in os.listdir(txt_folder) if f.endswith('.txt')]
    print('number of text files:', len(txt_files))

    results = []
    for file_name in sorted(txt_files):
        file_path = os.path.join(txt_folder, file_name)
        # ファイルを読み込む
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        try:
            tnm_stage = chain.run({"content": content}).strip()

            # TNM分類の形式を検証・修正（この部分はユーザーのvalidate_and_correct_tnm_output関数を流用）
            tnm_stage = validate_and_correct_tnm_output(tnm_stage)

            # TNM分類を分割
            tnm_parts = tnm_stage.split()
            if len(tnm_parts) >= 3:
                results.append({
                    "id": file_name.split('.')[0],
                    "t": tnm_parts[0],
                    "n": tnm_parts[1],
                    "m": tnm_parts[2],
                })
            else:
                print(f"ファイル {file_name} のTNM分類の出力形式が正しくありません: {tnm_stage}")
        except Exception as e:
            print(f"エラーが発生しました: {file_name} - {e}")
            continue

    # DataFrameに変換してCSVファイルに保存
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_csv, index=False)

    print(f"結果が{output_csv}に保存されました！")


モデル o1-preview-2024-09-12 を使用してTNM分類を予測中...
number of text files: 54
結果が../fewshot_submission_o1-preview-2024-09-12.csvに保存されました！
CPU times: user 1.13 s, sys: 52.8 ms, total: 1.18 s
Wall time: 41min 2s


In [ ]:
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

# モデルとトークナイザーのロード
model_name = "bigscience/bloomz-7b1"  # Hugging Faceモデル名を指定
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# テストデータのディレクトリ
txt_folder = '../radnlp_2024_train_val_20240731/ja/main_task/val/'
txt_files = [f for f in os.listdir(txt_folder) if f.endswith('.txt')]
print('number of text files:', len(txt_files))

# Few-shot例を定義
examples = [
    {"content": "対象文1", "tnm": "T1a N0 M0"},
    {"content": "対象文2", "tnm": "T2b N1 M0"},
]

# Few-shotプロンプトを構築
def construct_few_shot_prompt(examples, tnm_prompt_text):
    """
    Few-shot学習用のプロンプトを生成。
    """
    example_template = """
文章: {content}
出力: {tnm}
"""
    few_shot_examples = "\n".join(
        example_template.format(content=ex["content"], tnm=ex["tnm"]) for ex in examples
    )
    
    prompt_template = f"""
あなたは優秀な医師です。以下の文章に基づき肺癌に関して常に正しい判断ができます。
進行度分類は以下のTNM第８版に準拠しています。何も言わずに以下を覚え、与えられた文章からよく考えてTNM分類を選んでください。

{tnm_prompt_text}

Few-shot Examples:
{few_shot_examples}

以下の文章を読んで、TNM分類を正確に選択し、必ず以下の形式で出力してください：
T<number>[optional_letter] N<number>[optional_letter] M<number>[optional_letter]

文章:
"""
    return prompt_template

# tnm_prompt.txtを読み込み
with open('../tnm_prompt.txt', 'r', encoding='utf-8') as file:
    tnm_prompt_text = file.read()

# Few-shotプロンプトを生成
prompt_template = construct_few_shot_prompt(examples, tnm_prompt_text)

results = []

# 各テキストファイルを処理
for file_name in sorted(txt_files):
    file_path = os.path.join(txt_folder, file_name)
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # プロンプトを作成
    prompt = f"{prompt_template}{content}\n\n出力："
    
    # モデル推論の実行
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
    outputs = model.generate(**inputs, max_length=128)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    
    try:
        # 出力を分割し、TNM分類を抽出
        tnm_parts = output_text.split()
        if len(tnm_parts) >= 3:
            results.append({
                "id": file_name.split('.')[0],
                "t": tnm_parts[0],
                "n": tnm_parts[1],
                "m": tnm_parts[2],
            })
        else:
            print(f"ファイル {file_name} の出力形式が正しくありません: {output_text}")
    except Exception as e:
        print(f"エラーが発生しました: {file_name} - {e}")
        continue

# DataFrameに変換してCSVファイルに保存
results_df = pd.DataFrame(results)
output_csv = '../huggingface_tnm_classification_results.csv'
results_df.to_csv(output_csv, index=False)

print(f"結果が{output_csv}に保存されました！")
